# Finding Almanac Events with Skyfield
## Summary

Here are a series of functions that will find all instances of the following between a given start and end date.

* Solstices and equinoxes of Earth
* Moon phases
* Transits and anti-transits of any body
* Risings and settings of any body
* Beginning and end of civil, nautical, and astronomical twilight

The data at the Naval Observatory website are given to the nearest minute, so it's hard to tell exactly how close these numbers are. Occasionally the numbers calculated here round in the wrong direction; I'm not sure if it is because of different ephemerides, or if it is a problem with the algorithms.

### A high level overview of the 3 kinds of functions:
1. Semiprivate functions that are able to find zeroes and extrema in scalar functions
2. Semiprivate scalar functions for right ascension, declination, ecliptic longitude, altitude, etc.
3. Public functions that submit the scalar functions to the finder functions along with arguments the finder functions need to find the desired phenomena.

## The Code:

In [45]:
from skyfield.api import load
from skyfield.api import Topos
import scipy as sp
import scipy.optimize as optimize
import numpy as np
from functools import partial
from pprint import pprint
import math
from numbers import Number

planets = load('de421.bsp')
earth = planets['earth']
moon = planets['moon']
sun = planets['sun']
mars = planets['mars']
greenwich = earth + Topos('51.5 N', '0 W')
north_pole = earth + Topos('90 N', '0 W')
ts = load.timescale()

## `find_value`, A Value-Finding Algorithm

In [46]:
def _find_value(f, value, time_range, partition_width, slope_at_zero='positive', xtol=1e-10):
    if slope_at_zero == 'positive':
        def g(t):
            return (f(t) - value + 180) % 360 - 180                         # Steps 1 & 2
    elif slope_at_zero == 'negative':
        def g(t):
            return ((f(t) - value + 180) % 360 - 180) * -1                  # Steps 1, 2, % 3

    start = min(time_range.tt)                                              # Step 4
    end = max(time_range.tt)

    if isinstance(partition_width, Number):                                 # Step 5A
        partition_edges = list(np.arange(start, end, partition_width))
        partition_edges.append(end)
    elif isinstance(partition_width, list):                                 # Step 5B
        partition_edges = partition_width + [start, end]
        partition_edges.sort()

    result_times = []
    
    for i, left_edge in enumerate(partition_edges[:-1]):                    # Step 6
        
        right_edge = partition_edges[i+1]

        if g(left_edge) < 0 and g(right_edge) > 0:                          # Step 7
            result_time = sp.optimize.brentq(g, left_edge, right_edge, xtol=xtol)    # Step 8
            result_times.append(result_time)                                # Step 9

    return ts.tt(jd=result_times)                                           # Step 10

The arguments for this function are:
* `f`, the function in which to find `value`. This function should have a single scalar input, and give a single scalar output. I've found the best way to do this is to have it take a terrestrial time in jd format, and return an angle in degrees.
* `value`, the value to be found in `f`
* `time_range`, a date array. `find_value` searches between the earliest and latest dates in the array
* `partition_width`, see below
* `positive_slope`, see below
* `xtol`, x tolerance for zero finding algorithm. Defaults to 1e-9 days, or .000087 seconds.

The algorithm performs the following steps:
1. Transpose `f` so that `f(value)` becomes `f(0)`
2. Change the range of `f` from (0 to 360 degrees) to (-180 to +180 degrees)
3. If `slope_at_zero` is `negative`, multiply `f` by -1. This is because the algorithm can only finds zeroes with positive slope. Multiplying by -1 transforms the zeroes with negative slope into zeroes with positive slope.
4. Define `start` as `min(time_range)` and `end` as `max(time_range)`
5. Partition `f`:
 1. If `partition_width` is a `Number`: Partition `f` into partitions using the specified `partition_width` from `start` to `end`. When `f` is a constantly increasing function such as LHA, RA, or Ecliptic Longitude, `partition_width` should be slightly smaller than half the period of `f` so that the partitions cannot include both a zero and a discontinuity. `np.arange` is used because it can take floats as arguments. `end` is added only if it is not included by `np.arange` so that it can't be in the list twice.
 2. If `(partition_width` is a `list`: in some cases, two zeroes may be a very close to each other, thus possibly ending up in the same partition. The partition edges can be precalculated to avoid this and supplied to `partition_width` as a list of terrestrial times in jd format. `start` is added to the list as the beginning of the first partition, and `end` is added as the end of the last partition
6. For each partition, look at the `left_edge` and the `right_edge`
7. If the `left_edge` is negative, and the `right_edge` is positive, there must be a zero somewhere in the partition.
8. Submit the partition to the zero finding algorithm from scipy.
9. Append the result to the list of zeroes
10. Convert the list of terrestrial times to a date array and return it

## `find_extrema`, An Extrema-Finding Algorithm

In [47]:
def _find_extrema(f, time_range, partition_width, find='min', xtol=1e-10):
    if find == 'min':
        g = f                                                                # Skips steps 1-3
    elif find == 'max':
        def g(t):
            return f(t) * -1                                                 # Step 3

    start = min(time_range.tt)                                               # Step 4
    end = max(time_range.tt)

    partition_edges = list(np.arange(start, end, partition_width))           # Step 5A
    partition_edges.append(end)

    result_times = []
    
    for i, left_edge in enumerate(partition_edges[:-1]):                     # Step 6
        
        right_edge = partition_edges[i+1]

        if sp.misc.derivative(g, left_edge) < 0 and sp.misc.derivative(g, right_edge) > 0:   # Step 7
            result_time = sp.optimize.minimize_scalar(g, bracket=[left_edge, right_edge], options={'xtol': xtol}) # Step 8
            result_times.append(result_time['x'])                            # Step 9

    return ts.tt(jd=result_times)                                            # Step 10

This algorithm is exactly the same as `find_value`, except that:
* Steps 1, 2, and 4B are unneccesary
* Instead of testing that `left_edge` and `right_edge` are negative and positive, it tests that the derivative at `left_edge` and `right_edge` are negative and positive
* The partitions are submitted to a minimum-finding algorithm instead of a zero-finding algorithm
* `scipy.minimize_scalar` finds minimums only, so if you want to find a maximum you must set the keyword argument `find='max'`, just as `find_value` finds zeroes with positive slope by default, and if you want zeroes with negative slope you must set `slope_at_zero='negative'`

# Useful Scalar Functions

Functions that are going to be submitted to `find_value` or `find_extrema` should accept a singe scalar argument and return a single scalar. I found it best to have these functions take a single terrestrial time in jd format as an argument, and return an angle in degrees.

In [48]:
def _ra(body, t):
    """Returns the right ascension of 'body' in degrees at 
    terrestrial time t
    """
    return earth.at(ts.tt(jd=t)).observe(body).apparent().radec(epoch='date')[0]._degrees

_sun_ra = partial(_ra, sun)

def _ecliptic_lon(body, t):
    """Returns the ecliptic longitude  of body in degrees at terrestrial
    time t.
    """
    return earth.at(ts.tt(jd=t)).observe(body).apparent().ecliptic_latlon()[1]._degrees

def _moon_sun_lon_diff(t):
    """Returns the difference between the sun's and the moon's ecliptic
    longitude in degrees at terrestrial time t.
    """
    return _ecliptic_lon(moon, t) - _ecliptic_lon(sun, t)
    
def _local_sidereal(place, t):
    """Returns place's local apparent sidereal time at t in degrees"""
    return ts.tt(jd=t).gast*15 + place.positives[-1].longitude.degrees

def _lha(place, body, t):
    """Returns body's local hour angle at place at terrestrial time t 
    in degrees.
    """
    return _local_sidereal(place, t) - _ra(body, t)
    
def _topocentric_alt(place, body, t):
    """Returns body's topocentric altitude at place at terrestrial time
    t in degrees.
    """
    return place.at(ts.tt(jd=t)).observe(body).apparent().altaz()[0].degrees
    
def _geocentric_alt(place, body, t):
    """Returns body's geocentric altitude at place at terrestrial time
    t in degrees.
    """
    earth_sun = earth.at(ts.tt(jd=t)).observe(body)
    earth_place = place.positives[-1].at(ts.tt(jd=t))
    return 90 - earth_sun.separation_from(earth_place).degrees

def _moon_ul_topo_alt(place, t):
    """Returns topocentric altitude of the moon's upper limb at 
    place at terrestrial time t in degrees.
    """
    moon_distance = moon.positives[-1].at(ts.tt(jd=t)).distance().au
    moon_radius = 1.161781e-5 # 1738 km converted to au
    moon_sd = math.degrees(math.atan(moon_radius/moon_distance))
    return _topocentric_alt(place, moon, t) + moon_sd

def _moon_ul_geo_alt(place, t):
    """Returns geocentric altitude of the moon's upper limb at 
    place at terrestrial time t in degrees.
    """
    moon_distance = moon.positives[-1].at(ts.tt(jd=t)).distance().au
    moon_radius = 1.161781e-5 # 1738 km converted to au
    moon_sd = math.degrees(math.atan(moon_radius/moon_distance))
    return _geocentric_alt(place, moon, t) + moon_sd

A function like `ra` above cannot be submitted to scipy.minimize_scalar, because it has multiple arguments. However functools.partial can be used to create versions of functions that have certain arguments fixed to certain values. For example, `sun_ra` behaves like `ra`, but the value of the `body` argument is fixed as `sun`. This function `sun_ra` can  be submitted to scipy.minimize_scalar.

# Public Functions

## Finding Equinoxes and Solstices

In [49]:
def vernal_equinoxes(time_range):
    return _find_value(_sun_ra, 0, time_range, 365*.45)
    
def summer_solstices(time_range):
    return _find_value(_sun_ra, 90, time_range, 365*.45)
    
def autumnal_equinoxes(time_range):
    return _find_value(_sun_ra, 180, time_range, 365*.45)
    
def winter_solstices(time_range):
    return _find_value(_sun_ra, 270, time_range, 365*.45)
    
def solstices(time_range):
    winter = list(winter_solstices(time_range).tt)
    summer = list(summer_solstices(time_range).tt)  
    both = winter + summer
    both.sort()
    return ts.tt(jd=both)
    
def equinoxes(time_range):
    vernal = list(vernal_equinoxes(time_range).tt)
    autumnal = list(autumnal_equinoxes(time_range).tt)  
    both = vernal + autumnal
    both.sort()
    return ts.tt(jd=both)

In [50]:
result = vernal_equinoxes(ts.utc([2000, 2026]))
pprint(result.utc_jpl())

['A.D. 2000-Mar-20 07:35:16.9011 UT',
 'A.D. 2001-Mar-20 13:30:38.5505 UT',
 'A.D. 2002-Mar-20 19:16:08.7571 UT',
 'A.D. 2003-Mar-21 00:59:49.9711 UT',
 'A.D. 2004-Mar-20 06:48:33.9972 UT',
 'A.D. 2005-Mar-20 12:33:32.3595 UT',
 'A.D. 2006-Mar-20 18:25:26.5024 UT',
 'A.D. 2007-Mar-21 00:07:29.1471 UT',
 'A.D. 2008-Mar-20 05:48:14.8420 UT',
 'A.D. 2009-Mar-20 11:43:32.7448 UT',
 'A.D. 2010-Mar-20 17:32:13.7834 UT',
 'A.D. 2011-Mar-20 23:20:36.8532 UT',
 'A.D. 2012-Mar-20 05:14:32.5349 UT',
 'A.D. 2013-Mar-20 11:01:53.3373 UT',
 'A.D. 2014-Mar-20 16:57:07.9441 UT',
 'A.D. 2015-Mar-20 22:45:09.4808 UT',
 'A.D. 2016-Mar-20 04:30:04.7716 UT',
 'A.D. 2017-Mar-20 10:28:39.8426 UT',
 'A.D. 2018-Mar-20 16:15:20.4063 UT',
 'A.D. 2019-Mar-20 21:58:32.9710 UT',
 'A.D. 2020-Mar-20 03:49:36.4024 UT',
 'A.D. 2021-Mar-20 09:37:31.7100 UT',
 'A.D. 2022-Mar-20 15:33:25.1269 UT',
 'A.D. 2023-Mar-20 21:24:20.5729 UT',
 'A.D. 2024-Mar-20 03:06:28.1221 UT',
 'A.D. 2025-Mar-20 09:01:21.1551 UT']


Official vernal equinox data: http://aa.usno.navy.mil/data/docs/EarthSeasons.php. The values calculated here for 2006, 2012, 2019, and 2021, round in the wrong direction, and so are one minute off from the official numbers. I'm not sure why that is, or if it is an acceptable error or not.

## Finding Lunar Phases

In [51]:
def new_moons(time_range):
    return _find_value(_moon_sun_lon_diff, 0, time_range, 29*.45)

def first_quarters(time_range):
    return _find_value(_moon_sun_lon_diff, 270, time_range, 29*.45)
    
def full_moons(time_range):
    return _find_value(_moon_sun_lon_diff, 180, time_range, 29*.45)
    
def last_quarters(time_range):
    return _find_value(_moon_sun_lon_diff, 90, time_range, 29*.45)

In [52]:
result = new_moons(ts.utc([2017, 2018]))
pprint(result.utc_jpl())

['A.D. 2017-Jan-28 00:07:00.7865 UT',
 'A.D. 2017-Feb-26 14:58:22.3184 UT',
 'A.D. 2017-Mar-28 02:57:12.8874 UT',
 'A.D. 2017-Apr-26 12:16:08.8288 UT',
 'A.D. 2017-May-25 19:44:27.4714 UT',
 'A.D. 2017-Jun-24 02:30:41.9131 UT',
 'A.D. 2017-Jul-23 09:45:34.2441 UT',
 'A.D. 2017-Aug-21 18:30:10.4599 UT',
 'A.D. 2017-Sep-20 05:29:52.3499 UT',
 'A.D. 2017-Oct-19 19:12:03.8825 UT',
 'A.D. 2017-Nov-18 11:42:08.0461 UT',
 'A.D. 2017-Dec-18 06:30:24.9191 UT']


Official moon phase data for 2017: http://aa.usno.navy.mil/cgi-bin/aa_phases.pl?year=2016&month=1&day=1&nump=50&format=p

## Meridian Transits

In [53]:
def meridian_transits(place, body, time_range):
    body_lha = partial(_lha, place, body)
    return _find_value(body_lha, 0, time_range, .45)
    
def anti_meridian_transits(place, body, time_range):
    body_lha = partial(_lha, place, body)
    return _find_value(body_lha, 180, time_range, .45)
    
def all_transits(place, body, time_range):
    a = list(anti_meridian_transits(place, body, time_range).tt)
    b = list(meridian_transits(place, body, time_range).tt)
    both = a + b
    both.sort()
    return ts.tt(jd=both)

In [54]:
result = meridian_transits(greenwich, sun, ts.utc(2017, 4, [3, 8]))
pprint(result.utc_jpl())

['A.D. 2017-Apr-03 12:03:12.4465 UT',
 'A.D. 2017-Apr-04 12:02:55.0179 UT',
 'A.D. 2017-Apr-05 12:02:37.7435 UT',
 'A.D. 2017-Apr-06 12:02:20.6432 UT',
 'A.D. 2017-Apr-07 12:02:03.7377 UT']


Official sun transit data: http://aa.usno.navy.mil/cgi-bin/aa_mrst2.pl?form=2&ID=AA&year=2017&month=4&day=3&reps=5&body=10&place=Greenwich&lon_sign=-1&lon_deg=0&lat_sign=1&lat_deg=51&lat_min=30&tz_sign=-1

You can also see the rise times of other bodies, such as mars:

In [55]:
result = meridian_transits(greenwich, mars, ts.utc(2017, 4, [3, 8]))
pprint(result.utc_jpl())

['A.D. 2017-Apr-03 14:11:23.9041 UT',
 'A.D. 2017-Apr-04 14:10:16.9856 UT',
 'A.D. 2017-Apr-05 14:09:10.2725 UT',
 'A.D. 2017-Apr-06 14:08:03.7643 UT',
 'A.D. 2017-Apr-07 14:06:57.4607 UT']


Official mars transit data: http://aa.usno.navy.mil/cgi-bin/aa_mrst2.pl?form=2&ID=AA&year=2017&month=4&day=3&reps=5&body=4&place=Greenwich&lon_sign=-1&lon_deg=0&lat_sign=1&lat_deg=51&lat_min=3&tz_sign=-1

## Geocentric and Topocentric Risings and Settings
The topocentric rising and setting times are what we generally think of when we think of rising and setting. I include geocentric rising and setting only because **the sunrise and sunset times tabulated in almanacs use geocentric altitude**. I don't know of any other use for a geocentric rising and setting. The rising and setting functions use transit and anti-transit times as the edges of the partitions. This ensures that if `body` peeks above the horizon just before noon and sets just after, the two zeros will still be in different partitions.

In [56]:
def topocentric_risings(place, body, time_range):    
    partition_edges = list(all_transits(place, body, time_range).tt)
    
    f = partial(_topocentric_alt, place, sun)
    value = -34/60
    
    if body.target == 10:             # if body == sun
        value = -50/60
    if body.target == 301:            # if body == moon
        f = partial(_moon_ul_topo_alt, place)
        
    return _find_value(f, value, time_range, partition_edges)
    
def topocentric_settings(place, body, time_range):    
    partition_edges = list(all_transits(place, body, time_range).tt)

    f = partial(_topocentric_alt, place, sun)
    value = -34/60
    
    if body.target == 10:             # if body == sun
        value = -50/60
    if body.target == 301:            # if body == moon
        f = partial(_moon_ul_topo_alt, place)
        
    return _find_value(f, value, time_range, partition_edges, slope_at_zero='negative')
    
def geocentric_risings(place, body, time_range):    
    partition_edges = list(all_transits(place, body, time_range).tt)
    
    f = partial(_geocentric_alt, place, sun)
    value = -34/60
    
    if body.target == 10:             # if body == sun
        value = -50/60
    if body.target == 301:            # if body == moon
        f = partial(_moon_ul_geo_alt, place)
        
    return _find_value(f, value, time_range, partition_edges)
    
def geocentric_settings(place, body, time_range):    
    partition_edges = list(all_transits(place, body, time_range).tt)
    
    f = partial(_topocentric_alt, place, sun)
    value = -34/60
    
    if body.target == 10:             # if body == sun
        value = -50/60
    if body.target == 301:            # if body == moon
        f = partial(_moon_ul_geo_alt, place)
        
    return _find_value(f, value, time_range, partition_edges, slope_at_zero='negative')

In [57]:
result = topocentric_risings(greenwich, moon, ts.utc(2017, 4, [3,8]))
pprint(result.utc_jpl())

['A.D. 2017-Apr-03 10:09:58.7117 UT',
 'A.D. 2017-Apr-04 11:11:50.8656 UT',
 'A.D. 2017-Apr-05 12:18:24.0834 UT',
 'A.D. 2017-Apr-06 13:27:18.2985 UT',
 'A.D. 2017-Apr-07 14:36:42.3820 UT']


Official moon rising data: http://aa.usno.navy.mil/cgi-bin/aa_mrst2.pl?form=2&ID=AA&year=2017&month=4&day=3&reps=5&body=11&place=&lon_sign=-1&lon_deg=&lon_min=&lon_sec=&lat_sign=1&lat_deg=51&lat_min=30&lat_sec=&height=&tz=&tz_sign=-1

Here is an interesting edge case:

In [58]:
result = geocentric_risings(north_pole, sun, ts.utc([2016,2017]))
pprint(result.utc_jpl())

['A.D. 2016-Mar-18 01:46:21.3120 UT']


As expected, the sun rises once at the north pole each year. However, the official rising time that day is 7:53, which is off by over 6 hours. That will need more investigation. Official data: http://aa.usno.navy.mil/cgi-bin/aa_mrst2.pl?form=2&ID=AA&year=2017&month=1&day=1&reps=365&body=10&place=&lon_sign=-1&lon_deg=&lon_min=&lon_sec=&lat_sign=1&lat_deg=90&lat_min=&lat_sec=&height=&tz=&tz_sign=-1

## Twilight Beginning and End

In [59]:
def start_twilight(place, time_range, kind='civil'):
    partition_edges = list(all_transits(place, sun, time_range).tt)
    
    f = partial(_geocentric_alt, place, sun)
    if kind == 'civil':
        value = -6
    elif kind == 'nautical':
        value = -12
    elif kind == 'astronomical':
        value = -18
    
    return _find_value(f, value, time_range, partition_edges)

def end_twilight(place, time_range, kind='civil'):
    partition_edges = list(all_transits(place, sun, time_range).tt)
    
    f = partial(_geocentric_alt, place, sun)
    if kind == 'civil':
        value = -6
    elif kind == 'nautical':
        value = -12
    elif kind == 'astronomical':
        value = -18
    
    return _find_value(f, value, time_range, partition_edges, slope_at_zero='negative')

In [60]:
result = start_twilight(greenwich, ts.utc(2017, 4, [3,8]))
pprint(result.utc_jpl())

['A.D. 2017-Apr-03 04:56:56.6024 UT',
 'A.D. 2017-Apr-04 04:54:37.0513 UT',
 'A.D. 2017-Apr-05 04:52:17.5887 UT',
 'A.D. 2017-Apr-06 04:49:58.2476 UT',
 'A.D. 2017-Apr-07 04:47:39.0613 UT']


Official twilight data (same link as transit data above): http://aa.usno.navy.mil/cgi-bin/aa_mrst2.pl?form=2&ID=AA&year=2017&month=4&day=3&reps=5&body=10&place=Greenwich&lon_sign=-1&lon_deg=0&lat_sign=1&lat_deg=51&lat_min=30&tz_sign=-1

## Possible Syntax
If you decide that fuctions like this are something you want in Skyfield, here is an idea for a possible syntax.

    earth.at(t).observe(sun).summer_solstices()
                            .winter_solstices()
                            .solstices()
                            .vernal_equinoxes()
                            .autumnal_equinoxes()
                            .equinoxes()
    
    earth.at(t).observe(moon).new_moons(sun)
                             .first_quarters(sun)
                             .full_moons(sun)
                             .last_quarters(sun)
    
    earth.at(t).observe(body).geocentric_risings(topos)
                             .geocentric_settings(topos)
    
    topos.at(t).observe(sun).twilight_starts()
                            .twilight_ends()
    
    topos.at(t).observe(body).topocentric_risings()
                             .topocentric_settings()